<a href="https://colab.research.google.com/github/eunsun53/AI_toy_project/blob/main/task1_train_CoAtNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
import cv2
import matplotlib.pyplot as plt
import pandas as pd 
import tensorflow as tf 
import torch 
from torch import nn 
import torch.nn.functional as F
import torch.optim as optim 
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
# !pip install coca-pytorch
# !pip install vit-pytorch>=0.35.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#드라이브-코랩 연동 
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
cd /content/gdrive/MyDrive/dataset/FASCODE2022/task1

/content/gdrive/MyDrive/dataset/FASCODE2022/task1


In [ ]:
# import pandas as pd
# from dataset import ETRIDataset_emo
# from networks import *

# df = pd.read_csv('/content/gdrive/MyDrive/dataset/FASCODE2022/task1_data/info_etri20_emotion_train.csv')
# train_dataset = ETRIDataset_emo(df, base_path='/content/gdrive/MyDrive/dataset/FASCODE2022/task1_data/train/')

# plt.imshow(train_dataset[0]['ori_image'])

In [ ]:
# df[0:1]

In [ ]:
# df2 = pd.DataFrame(columns=list(df.columns))
# df2.loc[0] = [0,0,0,0,0,0,0,0,0]
# df2.loc[1] = [1, 1, 1, 1, 1, 1, 1, 1, 1]
# df2.loc[2]= [2, 1, 1, 1, 1, 1, 1, 1, 1]
# df2.loc[3]= [3, 1, 1, 1, 1, 1, 1, 1, 1]
# df2.loc[4]= [4, 1, 1, 1, 1, 1, 1, 1, 1]


# df2 =df2.drop(1)

# df2 =df2.drop(3)
# df2

In [ ]:
# df2.iloc[1]

In [ ]:
# print(len(train_dataset))

In [ ]:
# test_df = pd.DataFrame(columns=list(df.columns))
# idx = 0
# for i in range(1000):
#   test_df.loc[i] = df.iloc[10*i]
#   idx += 1 
# idx = 0
# for i in range(1000):
#   df =df.drop(10*i)
#   idx += 1
  
# test_df.to_csv('/content/gdrive/MyDrive/dataset/FASCODE2022/task1_data/info_etri20_emotion_test.csv', index=False)


In [ ]:
# df.to_csv('/content/gdrive/MyDrive/dataset/FASCODE2022/task1_data/info_etri20_emotion_train.csv', index=False)


In [ ]:
# print("df ",len(df), "test", len(test_df))

In [ ]:
# train_dataset[0]['ori_image'].shape

In [ ]:
# from torchvision import transforms
# ori = train_dataset[0]['ori_image']
# to_tensor = transforms.ToTensor()
# ori_tensor = to_tensor(ori)

In [ ]:
# plt.imshow(ori_tensor.permute(1, 2, 0)  )

In [ ]:
# from torchvision import transforms
# from skimage import io, transform, color

# image_ = train_dataset[0]['image']

In [ ]:
# plt.imshow(image_.permute(1, 2, 0)  )

In [ ]:
# import torchvision.transforms.functional as F
# ori = train_dataset[0]['image']
# new_ = F.to_pil_image(ori)
# plt.imshow(np.asarray(new_))

In [ ]:
# print(np.array(new_))

In [ ]:
# image_.shape

In [ ]:
# print(image_.dtype)

In [5]:
import easydict

a = easydict.EasyDict({
        "version" : 'Baseline_ResNet_emo',
        "epochs" : 100,
        "lr" : 1e-4,
        "batch_size" :128,
        "seed" : None
    })

In [7]:
from dataset import ETRIDataset_emo
from networks_CoAtNet import *

import pandas as pd
import os
import argparse
import time

import torch
import torch.utils.data
import torch.utils.data.distributed

# parser = argparse.ArgumentParser()
# parser.add_argument("--version", type=str, default='Baseline_ResNet_emo')
# parser.add_argument('--epochs', default=100, type=int, metavar='N',
#                     help='number of total epochs to run')
# parser.add_argument('--lr', default=0.0001, type=float, metavar='N',
#                     help='learning rate')
# parser.add_argument('-b', '--batch-size', default=128, type=int,
#                     metavar='N',
#                     help='mini-batch size (default: 256), this is the total '
#                          'batch size of all GPUs on the current node when '
#                          'using Data Parallel or Distributed Data Parallel')
# parser.add_argument('--seed', default=None, type=int,
#                     help='seed for initializing training. ')

# a = parser.parse_args()

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def main():
    """ The main function for model training. """
    if os.path.exists('models') is False:
        os.makedirs('models')

    save_path = 'models/' + a.version
    if os.path.exists(save_path) is False:
        os.makedirs(save_path)

    net = Baseline_ResNet_emo().to(DEVICE)

    df = pd.read_csv('/content/gdrive/MyDrive/dataset/FASCODE2022/task1/info_etri20_emotion_train.csv')
    train_dataset = ETRIDataset_emo(df, base_path='/content/gdrive/MyDrive/dataset/FASCODE2022/task1/train/')
    
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=a.batch_size, shuffle=True, num_workers=0)

    optimizer = torch.optim.Adam(net.parameters(), lr=a.lr)
    criterion = nn.CrossEntropyLoss().to(DEVICE)

    total_step = len(train_dataloader)
    step = 0
    t0 = time.time()

    for epoch in range(a.epochs):
        net.train()

        for i, sample in enumerate(train_dataloader):
            optimizer.zero_grad()
            step += 1
            for key in sample:
                sample[key] = sample[key].to(DEVICE)

            out_daily, out_gender, out_embel = net(sample)

            loss_daily = criterion(out_daily, sample['daily_label'])
            loss_gender = criterion(out_gender, sample['gender_label'])
            loss_embel = criterion(out_embel, sample['embel_label'])
            loss = loss_daily + loss_gender + loss_embel

            loss.backward()
            optimizer.step()

            if (i + 1) % 10 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, '
                      'Loss_daily: {:.4f}, Loss_gender: {:.4f}, Loss_embel: {:.4f}, Time : {:2.3f}'
                      .format(epoch + 1, a.epochs, i + 1, total_step, loss.item(),
                              loss_daily.item(), loss_gender.item(), loss_embel.item(), time.time() - t0))

                t0 = time.time()

        if ((epoch + 1) % 10 == 0):
            a.lr *= 0.9
            optimizer = torch.optim.Adam(net.parameters(), lr=a.lr)
            print("learning rate is decayed")


        if ((epoch + 1) % 10 == 0):
            print('Saving Model....')
            torch.save(net.state_dict(), save_path + '/model_' + str(epoch + 1) + '.pkl')
            print('OK.')


if __name__ == '__main__':
    main()



RuntimeError: ignored